In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jax import jit, grad
import qcsys as qs
import jax.numpy as jnp
import jaxquantum as jqt

In [3]:
def calculate_chi_diagonalize():
    _, Ec_a, El_a = qs.calculate_lambda_over_four_resonator_zpf(5, 50)

    resonator = qs.Resonator.create(
        10,
        {"Ec": Ec_a, "El": El_a},
        N_pre_diag=10,
    )

    Ec_q = 1
    El_q = 0.5
    Ej_q = 8

    qubit = qs.Fluxonium.create(
        25,
        {"Ec": Ec_q, "El": El_q, "Ej": Ej_q, "phi_ext": 0.47},
        use_linear=False,
        N_pre_diag=100,
    )

    g_rq = 0.3

    devices = [resonator, qubit]
    r_indx = 0
    q_indx = 1
    Ns = [device.N for device in devices]

    a0 = qs.promote(resonator.ops["a"], r_indx, Ns)
    a0_dag = qs.promote(resonator.ops["a_dag"], r_indx, Ns)

    q0 = qs.promote(qubit.ops["a"], q_indx, Ns)
    q0_dag = qs.promote(qubit.ops["a_dag"], q_indx, Ns)

    couplings = []
    couplings.append(-g_rq * (a0 - a0_dag) @ (q0 - q0_dag))

    system = qs.System.create(devices, couplings=couplings)
    system.params["g_rq"] = g_rq

    Es, kets = system.calculate_eig()

    # chi ----
    χ_e = Es[1:, 1] - Es[:-1, 1]
    χ_g = Es[1:, 0] - Es[:-1, 0]
    χ = χ_e - χ_g

    return χ

In [4]:
calculate_chi_diagonalize()

Array([-0.00399307, -0.00369734, -0.0191641 ,  0.13341733, -0.19415304,
       -0.00187664, -1.06201788,  1.10409898, -0.49863942], dtype=float64)

# Normal Mode Approach

In [7]:
# Normal Mode Estimate

def calculate_chi_normal_modes():
    _, Ec_a, El_a = qs.calculate_lambda_over_four_resonator_zpf(5, 50)

    resonator = qs.Resonator.create(
        10,
        {"Ec": Ec_a, "El": El_a},
        N_pre_diag=10,
    )

    Ec_q = 1
    El_q = 0.5
    Ej_q = 8

    qubit = qs.Fluxonium.create(
        25,
        {"Ec": Ec_q, "El": El_q, "Ej": Ej_q, "phi_ext": 0.47},
        use_linear=True,
        N_pre_diag=100,
    )

    g_rq = 0.3

    devices = [resonator, qubit]
    r_indx = 0
    q_indx = 1
    Ns = [device.N for device in devices]

    a0 = qs.promote(resonator.ops["a"], r_indx, Ns)
    a0_dag = qs.promote(resonator.ops["a_dag"], r_indx, Ns)

    q0 = qs.promote(qubit.ops["a"], q_indx, Ns)
    q0_dag = qs.promote(qubit.ops["a_dag"], q_indx, Ns)

    couplings = []
    couplings.append(-g_rq * (a0 - a0_dag) @ (q0 - q0_dag))

    linear_system = qs.System.create(devices, couplings=couplings)
    linear_system.params["g_rq"] = g_rq

    Es, kets = linear_system.calculate_eig()

    # participations

    vac = kets[0, 0]
    
    e = {}
    e[linear_system.devices[0].label] = kets[1, 0]
    e[linear_system.devices[1].label] = kets[0, 1]
    e.keys()

    """extract participations"""
    ϕ = {device.label: {} for device in linear_system.devices}

    for j, d1 in enumerate(linear_system.devices):
        phi_0 = linear_system.promote(d1.ops["phi"], j).data
        for d2 in linear_system.devices:
            ϕ[d1.label][d2.label] = jqt.dag_data(e[d2.label]) @ phi_0 @ vac

    theta_a = ϕ["Fluxonium0"]["Resonator0"]


    return theta_a

In [8]:
calculate_chi_normal_modes()

Array(0.19994616+0.j, dtype=complex128)